In [27]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [28]:
node = "lotus"

In [29]:
#import exp_filtered
data = pd.read_csv('../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
288,745,2024-01-19 20:52:52.8522174,2024-01-19 21:22:52.8564323,general,random27,8,1,23,4,8,2,0.25,3.0,1705697572,1705699372
289,746,2024-01-19 21:26:27.3234593,2024-01-19 21:56:27.3265222,general,random27,8,1,23,4,8,2,0.25,3.0,1705699587,1705701387
290,747,2024-01-19 22:00:01.6380175,2024-01-19 22:30:01.6457071,general,random28,20,18,23,8,4,16,0.50,1.0,1705701601,1705703401
294,751,2024-01-20 00:14:24.4097982,2024-01-20 00:44:24.4126883,general,random29,18,20,8,4,6,4,0.25,3.0,1705709664,1705711464
295,752,2024-01-20 00:47:58.7269655,2024-01-20 01:17:58.7312773,general,random29,18,20,8,4,6,4,0.25,3.0,1705711678,1705713478
296,753,2024-01-20 01:21:32.6299318,2024-01-20 01:51:32.6406703,general,random30,15,4,20,8,2,8,1.00,1.0,1705713692,1705715492
299,758,2024-01-20 04:09:25.0644731,2024-01-20 04:39:25.0711445,general,random31,8,6,18,4,4,4,0.25,0.5,1705723765,1705725565
301,760,2024-01-20 05:16:34.1638368,2024-01-20 05:46:34.1666149,general,random32,3,2,5,2,2,2,0.50,1.0,1705727794,1705729594
302,761,2024-01-20 05:50:08.2461471,2024-01-20 06:20:08.2541699,general,random32,3,2,5,2,2,2,0.50,1.0,1705729808,1705731608
303,762,2024-01-20 06:23:42.6007382,2024-01-20 06:53:42.6043583,general,random33,19,8,23,4,8,2,0.25,3.0,1705731822,1705733622


In [30]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/mess_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,type,experiment
0,2024-01-20 06:30:54.319704+00:00,11.0,762
1,2024-01-20 06:30:54.336238+00:00,11.0,762
2,2024-01-20 06:30:54.336261+00:00,11.0,762
3,2024-01-20 06:30:54.336267+00:00,11.0,762
4,2024-01-20 06:30:54.336271+00:00,11.0,762
5,2024-01-20 06:30:54.336284+00:00,11.0,762
6,2024-01-20 06:30:54.336289+00:00,11.0,762
7,2024-01-20 06:30:54.433203+00:00,11.0,762
8,2024-01-20 06:30:54.442474+00:00,11.0,762
9,2024-01-20 06:30:54.444582+00:00,11.0,762


In [31]:
# Transpose the table -> type and count
df = messages#.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='5s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="5s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_82388/2738252698.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:00+00:00,345.0,56.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,466.0,73.0,0.0,0.0,0.0,64.0
2,390,2023-10-04 18:52:10+00:00,781.0,124.0,0.0,0.0,0.0,64.0
3,390,2023-10-04 18:52:15+00:00,734.0,111.0,0.0,0.0,0.0,32.0
4,390,2023-10-04 18:52:20+00:00,554.0,91.0,0.0,0.0,0.0,64.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,776.0,125.0,0.0,0.0,0.0,64.0
96,390,2023-10-04 19:00:00+00:00,479.0,74.0,0.0,0.0,0.0,32.0
97,390,2023-10-04 19:00:05+00:00,820.0,129.0,0.0,0.0,0.0,64.0
98,390,2023-10-04 19:00:10+00:00,821.0,134.0,0.0,0.0,0.0,64.0


In [32]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

# eventsTS.to_csv('./events20s.csv')
# eventsTS.head(500)

In [33]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

query_mess = pd.DataFrame()
query_pub = pd.DataFrame()

for index, row in experiments.iterrows():
    if os.path.exists('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_mess = pd.DataFrame(data)
        deliverMessage = pd.concat([query_mess, deliverMessage])
        del query_mess

    if os.path.exists('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_pub = pd.DataFrame(data)
        publishMessage = pd.concat([publishMessage, query_pub])
        del query_pub

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-01-20 06:30:57.189007+00:00,ACQIARIgmnEb0qZvegt7jQw9GezNHzgwSRDnEXJHLT5U75...,762
1,2024-01-20 06:30:57.279094+00:00,ACQIARIgZTwM9zqvuGbntFGs3WCNz/6u9RqIjPBSg9JdSf...,762
2,2024-01-20 06:30:57.281353+00:00,ACQIARIg4OtKsZMdIo2i8pZSqW+IQGQGykQif/q8FoDTVF...,762
3,2024-01-20 06:30:57.332667+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...,762
4,2024-01-20 06:30:57.392188+00:00,ACQIARIg2E5YMFznHacBOytrj+HcCNQBtpgMqFDEHtnRlQ...,762
5,2024-01-20 06:30:57.432512+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,762
0,2024-01-20 05:57:20.584045+00:00,ACQIARIg3KUU2uoJavOyzcVNRJLjO/l6ByIRKxWBd/gS56...,761
0,2024-01-20 05:23:47.268661+00:00,ACQIARIgFEGM3FOQ2H79LXNaPHJ5b+cZHWLYTjh+h8ID3c...,760
1,2024-01-20 05:23:47.298049+00:00,ACQIARIg48RRwjv88DzyjXOEIFC0zdxIAsXkF9DSfh2nGS...,760
0,2024-01-20 04:16:37.751160+00:00,ACQIARIg9J81Yj/smCNF6/9o0XZmWCIQLH5scTrDUQ/5Fy...,758


In [34]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.978680+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.978989+00:00,309
1,2023-10-04 18:52:04.979101+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.979353+00:00,252
2,2023-10-04 18:52:04.979467+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.979741+00:00,274
3,2023-10-04 18:52:04.979869+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.980060+00:00,191
4,2023-10-04 18:52:04.980205+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.980706+00:00,501
5,2023-10-04 18:52:04.980533+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.980758+00:00,225
6,2023-10-04 18:52:07.987652+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.988025+00:00,373
7,2023-10-04 18:52:07.988149+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.988422+00:00,273
8,2023-10-04 18:52:07.988543+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.989014+00:00,471
9,2023-10-04 18:52:07.988927+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.989328+00:00,401


In [35]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='5s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_82388/2407133802.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:00+00:00,292.000000
1,390,2023-10-04 18:52:05+00:00,373.333333
2,390,2023-10-04 18:52:10+00:00,649.083333
3,390,2023-10-04 18:52:15+00:00,337.666667
4,390,2023-10-04 18:52:20+00:00,234.000000
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,391.166667
96,390,2023-10-04 19:00:00+00:00,430.333333
97,390,2023-10-04 19:00:05+00:00,404.166667
98,390,2023-10-04 19:00:10+00:00,690.333333


In [36]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,391.166667,776.0,125.0,0.0,0.0,0.0,64.0
96,390,2023-10-04 19:00:00+00:00,430.333333,479.0,74.0,0.0,0.0,0.0,32.0
97,390,2023-10-04 19:00:05+00:00,404.166667,820.0,129.0,0.0,0.0,0.0,64.0
98,390,2023-10-04 19:00:10+00:00,690.333333,821.0,134.0,0.0,0.0,0.0,64.0


In [37]:
rpc = pd.DataFrame()

for idx, exec in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/rpc_'+node+'_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.135930+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135962+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.139095+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.139121+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.139132+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.139214+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.139248+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.139264+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.139276+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.139293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [38]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='5s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_82388/1580870231.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [39]:
message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='5s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_82388/1432395461.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:00+00:00,56
1,390,2023-10-04 18:52:05+00:00,73
2,390,2023-10-04 18:52:10+00:00,124
3,390,2023-10-04 18:52:15+00:00,111
4,390,2023-10-04 18:52:20+00:00,91
5,390,2023-10-04 18:52:25+00:00,124
6,390,2023-10-04 18:52:30+00:00,105
7,390,2023-10-04 18:52:35+00:00,113
8,390,2023-10-04 18:52:40+00:00,127
9,390,2023-10-04 18:52:45+00:00,91


In [40]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='5s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_82388/448925867.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,452
1,390,2023-10-04 18:52:05+00:00,638
2,390,2023-10-04 18:52:10+00:00,1050
3,390,2023-10-04 18:52:15+00:00,967
4,390,2023-10-04 18:52:20+00:00,757
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,1046
96,390,2023-10-04 19:00:00+00:00,637
97,390,2023-10-04 19:00:05+00:00,1098
98,390,2023-10-04 19:00:10+00:00,1106


In [41]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,452
1,390,2023-10-04 18:52:05+00:00,638
2,390,2023-10-04 18:52:10+00:00,1050
3,390,2023-10-04 18:52:15+00:00,967
4,390,2023-10-04 18:52:20+00:00,757
5,390,2023-10-04 18:52:25+00:00,1018
6,390,2023-10-04 18:52:30+00:00,892
7,390,2023-10-04 18:52:35+00:00,893
8,390,2023-10-04 18:52:40+00:00,1066
9,390,2023-10-04 18:52:45+00:00,791


In [42]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipReceived'] = gossipBandwidth['count']
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/5
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,396,79.2
1,390,2023-10-04 18:52:05+00:00,565,113.0
2,390,2023-10-04 18:52:10+00:00,926,185.2
3,390,2023-10-04 18:52:15+00:00,856,171.2
4,390,2023-10-04 18:52:20+00:00,666,133.2
...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,921,184.2
96,390,2023-10-04 19:00:00+00:00,563,112.6
97,390,2023-10-04 19:00:05+00:00,969,193.8
98,390,2023-10-04 19:00:10+00:00,972,194.4


In [43]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/5
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,11.2
1,390,2023-10-04 18:52:05+00:00,14.6
2,390,2023-10-04 18:52:10+00:00,24.8
3,390,2023-10-04 18:52:15+00:00,22.2
4,390,2023-10-04 18:52:20+00:00,18.2
5,390,2023-10-04 18:52:25+00:00,24.8
6,390,2023-10-04 18:52:30+00:00,21.0
7,390,2023-10-04 18:52:35+00:00,22.6
8,390,2023-10-04 18:52:40+00:00,25.4
9,390,2023-10-04 18:52:45+00:00,18.2


In [44]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_82388/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2
...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:27:00+00:00,400.083333,1376.0,128.0,0.0,0.0,0.0,10.0,25.6
496,394,2023-10-04 21:27:05+00:00,559.583333,1274.0,118.0,0.0,0.0,0.0,10.0,23.6
497,394,2023-10-04 21:27:10+00:00,379.500000,891.0,82.0,0.0,0.0,0.0,10.0,16.4
498,394,2023-10-04 21:27:15+00:00,477.000000,1321.0,122.0,0.0,0.0,0.0,10.0,24.4


In [45]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/5

totalBand.head(500)

/tmp/ipykernel_82388/511213079.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/511213079.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2
...,...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:27:00+00:00,400.083333,1376.0,128.0,0.0,0.0,0.0,10.0,25.6,600.8
496,394,2023-10-04 21:27:05+00:00,559.583333,1274.0,118.0,0.0,0.0,0.0,10.0,23.6,555.4
497,394,2023-10-04 21:27:10+00:00,379.500000,891.0,82.0,0.0,0.0,0.0,10.0,16.4,389.0
498,394,2023-10-04 21:27:15+00:00,477.000000,1321.0,122.0,0.0,0.0,0.0,10.0,24.4,575.8


In [46]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_82388/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4,396.0,79.2
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8,565.0,113.0
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2,926.0,185.2
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2,856.0,171.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2,666.0,133.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:27:00+00:00,400.083333,1376.0,128.0,0.0,0.0,0.0,10.0,25.6,600.8,1500.0,300.0
496,394,2023-10-04 21:27:05+00:00,559.583333,1274.0,118.0,0.0,0.0,0.0,10.0,23.6,555.4,1385.0,277.0
497,394,2023-10-04 21:27:10+00:00,379.500000,891.0,82.0,0.0,0.0,0.0,10.0,16.4,389.0,972.0,194.4
498,394,2023-10-04 21:27:15+00:00,477.000000,1321.0,122.0,0.0,0.0,0.0,10.0,24.4,575.8,1436.0,287.2


In [47]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4,396.0,79.2
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8,565.0,113.0
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2,926.0,185.2
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2,856.0,171.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2,666.0,133.2
5,390,2023-10-04 18:52:25+00:00,381.500000,750.0,124.0,0.0,0.0,0.0,64.0,24.8,353.6,894.0,178.8
6,390,2023-10-04 18:52:30+00:00,328.166667,671.0,105.0,0.0,0.0,0.0,32.0,21.0,312.6,787.0,157.4
7,390,2023-10-04 18:52:35+00:00,349.722222,653.0,113.0,0.0,0.0,0.0,64.0,22.6,309.2,780.0,156.0
8,390,2023-10-04 18:52:40+00:00,581.083333,792.0,127.0,0.0,0.0,0.0,64.0,25.4,371.6,939.0,187.8
9,390,2023-10-04 18:52:45+00:00,473.166667,598.0,91.0,0.0,0.0,0.0,32.0,18.2,277.8,700.0,140.0


In [49]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../Datasets/consolidated5s_'+node+'.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,...,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4,...,79.2,unl,8,6,12,4,8,2,0.25,1.0
1,390,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8,...,113.0,unl,8,6,12,4,8,2,0.25,1.0
2,390,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2,...,185.2,unl,8,6,12,4,8,2,0.25,1.0
3,390,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2,...,171.2,unl,8,6,12,4,8,2,0.25,1.0
4,390,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2,...,133.2,unl,8,6,12,4,8,2,0.25,1.0
5,390,381.500000,750.0,124.0,0.0,0.0,0.0,64.0,24.8,353.6,...,178.8,unl,8,6,12,4,8,2,0.25,1.0
6,390,328.166667,671.0,105.0,0.0,0.0,0.0,32.0,21.0,312.6,...,157.4,unl,8,6,12,4,8,2,0.25,1.0
7,390,349.722222,653.0,113.0,0.0,0.0,0.0,64.0,22.6,309.2,...,156.0,unl,8,6,12,4,8,2,0.25,1.0
8,390,581.083333,792.0,127.0,0.0,0.0,0.0,64.0,25.4,371.6,...,187.8,unl,8,6,12,4,8,2,0.25,1.0
9,390,473.166667,598.0,91.0,0.0,0.0,0.0,32.0,18.2,277.8,...,140.0,unl,8,6,12,4,8,2,0.25,1.0
